In [1]:
import numpy as np
import copy
from multiprocessing import Pool
import multiprocessing
from scipy import stats

In [2]:
def init_pi(N_a,N_s):
    """
    function to generate policy,
    inputs:
        N_a - number of actions;
        N_s - number of states;
    outputs:
        pi(a|s) - np.array of shape N_a x N_s
    """
    np.random.seed(1453)
    Pi_matr = np.random.uniform(0.0,1.0,(N_a,N_s))
    norm_coef = Pi_matr.sum(axis=0)
    Pi_matr = Pi_matr / norm_coef.reshape((1,N_s))
    #check if stochastic
    print(Pi_matr.sum(axis=0))
    return Pi_matr

In [3]:
def generate_dynamics(N_a,N_s,b):
    """
    function to generate transition probabilities,
    inputs:
        N_a - number of actions;
        N_s - number of states;
        b - branching number
    outputs:
        pi(s'|s,a) - np.array of shape N_s x N_s x N_a
    """
    np.random.seed(1812)
    inds_nonzero = np.zeros((N_s,N_a,b),dtype = int)
    for i in range(N_s):
        for j in range(N_a):
            inds_nonzero[i,j] = np.random.choice(N_s, size=b, replace=False)
    Pi_matr = np.zeros((N_s,N_s,N_a),dtype=float)
    for i in range(N_s):
        for j in range(N_a):
            Pi_matr[inds_nonzero[i,j],i,j] = np.random.uniform(0.0,1.0,b)
    norm_coef = Pi_matr.sum(axis=0)
    Pi_matr = Pi_matr / norm_coef.reshape((1,N_s,N_a))
    print(Pi_matr.sum(axis=0))
    return Pi_matr,inds_nonzero

In [4]:
def state_transitions(P,pi):
    """
    function to generate transition probabilities,
    inputs:
        P(s'|s,a) - np.array of shape N_s x N_s x N_a, transition probabilities;
        pi(a|s) - np.array of shape N_a x N_s, policy;
    outputs:
        p(s'|s) - transition probability matrix of shape (N_s,N_s)
    """
    np.random.seed(1812)
    P_s = np.zeros((N_s,N_s),dtype = float)
    for i in range(N_s):
        for j in range(N_s):
            P_s[i,j] = np.dot(P[i,j,:],pi[:,j])
    return P_s 

In [5]:
def init_rewards(N_a,N_s):
    """
    function to generate rewards,
    inputs:
        N_a - number of actions;
        N_s - number of states;
    outputs:
        R(a,s) - np.array of rewards (shape N_a x N_s)  
    """
    np.random.seed(1821)
    R = Pi_matr = np.random.uniform(0.0,2.0,(N_a,N_s))
    return R

In [6]:
#global constants
N_a = 10
N_s = 50
#gamma = 0.99
gamma = 0.9
b = 2

In [7]:
#init policy matrix
Policy = init_pi(N_a,N_s)
#init transition matrix
P,Inds_nz = generate_dynamics(N_a,N_s,b)
#init rewards
R = init_rewards(N_a,N_s)
#init state transition matrix
S_trans = state_transitions(P,Policy)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1

### Solve system to find $\theta^*$ (i.e. true $V_{\pi}(s)$)

In [8]:
#system matrix
A = np.eye(N_s) - gamma*(S_trans.T)
#right hand side
b = np.sum(Policy*R,axis=0)
theta_star_old = np.linalg.inv(A) @ b
print(theta_star_old)

[9.84953413 9.52818882 9.10077358 9.20650432 9.46348305 9.53415339
 9.60132343 9.58224446 9.02248147 9.55440038 9.61889827 9.51152889
 9.24885002 9.77558082 9.80059676 9.89326659 9.40835507 9.31492265
 9.19517124 9.50103756 9.20002613 9.22017523 9.09961668 9.38826035
 9.25805059 9.07006076 9.32369281 9.5107984  9.51615943 9.50436866
 9.38564581 9.64467251 9.71316469 9.33079435 9.09258728 9.66603574
 9.11930009 9.36024236 9.35390133 9.24264092 9.57049198 9.25693893
 9.35382775 9.29097659 9.04836883 9.14591195 9.4543717  9.56269105
 9.68137285 9.68337974]


### Find stationary distribution over state space

In [9]:
#note that in my notations they are usual (right) eigenvalues
eigvals, eigfuncs = np.linalg.eig(S_trans)
print(eigvals)
pi_states = -np.real(eigfuncs[:,0])
pi_states = pi_states/np.sum(pi_states)
#pi_states - statinary distribution over states
print(pi_states)
print(np.sum(pi_states))

[ 1.        +0.j         -0.26430692+0.12903984j -0.26430692-0.12903984j
 -0.28404277+0.03470012j -0.28404277-0.03470012j -0.13168546+0.2269733j
 -0.13168546-0.2269733j   0.24202435+0.09985377j  0.24202435-0.09985377j
  0.13218168+0.22116056j  0.13218168-0.22116056j  0.0566535 +0.23111173j
  0.0566535 -0.23111173j  0.1641442 +0.1417069j   0.1641442 -0.1417069j
  0.21104999+0.j          0.19396104+0.06409327j  0.19396104-0.06409327j
  0.18542299+0.j          0.11909186+0.16312541j  0.11909186-0.16312541j
 -0.20534831+0.10750108j -0.20534831-0.10750108j -0.00321329+0.21593335j
 -0.00321329-0.21593335j -0.06451847+0.20039498j -0.06451847-0.20039498j
 -0.07494813+0.18662476j -0.07494813-0.18662476j  0.05846562+0.16833902j
  0.05846562-0.16833902j -0.20275598+0.j         -0.14950415+0.07123611j
 -0.14950415-0.07123611j -0.16728513+0.j          0.01340965+0.15919804j
  0.01340965-0.15919804j  0.12790264+0.j          0.07303261+0.09720718j
  0.07303261-0.09720718j -0.03112849+0.09440118j -0.0

### Find $\theta^*$ by another approach

In [10]:
A_star = np.zeros((N_s,N_s),dtype=float)
for i in range(N_s):
    for j in range(N_s):
        A_star[i,j] = pi_states[i]*S_trans[j,i]
A_star = np.diag(pi_states) - gamma*A_star
b_star = b*pi_states
theta_star_new = np.linalg.inv(A_star) @ b_star
#print("error between to True solutions: ",np.linalg.norm(theta_star-theta_star_new))

### Find $\theta^*$ for eligibility traces

In [11]:
#lambda parameter
TD_lambda = 0.9
#compute eigval decomposition
eigvals, eigfuncs = np.linalg.eig(S_trans.T)
B_matr_diag = np.diag(np.ones(N_s)/(1-TD_lambda*gamma*eigvals))
B_matr_diag_next = np.diag(eigvals/(np.ones(N_s)-TD_lambda*gamma*eigvals))
b_star_new = (np.diag(pi_states)).dot(eigfuncs.dot(B_matr_diag.dot(np.linalg.inv(eigfuncs))))@b
print(b_star)
print(np.real(b_star_new))
A_star_1 = (np.diag(pi_states)).dot(eigfuncs.dot(B_matr_diag.dot(np.linalg.inv(eigfuncs))))
A_star_2 = (np.diag(pi_states)).dot(eigfuncs.dot(B_matr_diag_next.dot(np.linalg.inv(eigfuncs))))
A_star_new = A_star_1 - gamma*A_star_2
A_star_new = np.real(A_star_new)
theta_star = np.linalg.inv(A_star_new) @ b_star_new
theta_star = np.real(theta_star)
print(theta_star)

[0.00808426 0.01285082 0.01170909 0.02176083 0.01588613 0.02483413
 0.0198603  0.01868572 0.01266242 0.0197913  0.02456966 0.02609498
 0.02172233 0.02238264 0.01548495 0.02801899 0.03122647 0.02391849
 0.0157511  0.01481018 0.02423274 0.00809741 0.01296728 0.01933507
 0.02217375 0.01602306 0.02167556 0.01733259 0.04120114 0.02482404
 0.01754035 0.02203904 0.0227591  0.01569568 0.00668252 0.01683935
 0.01265828 0.01151986 0.01287138 0.01240695 0.02104786 0.01163515
 0.01899798 0.00942113 0.01151103 0.02010333 0.01674638 0.02801735
 0.03190254 0.02185303]
[0.03285338 0.0637148  0.08175653 0.13536277 0.08264727 0.11355754
 0.09086673 0.09397152 0.09385352 0.08641578 0.10557799 0.1246464
 0.13709169 0.09264265 0.06494005 0.106938   0.15449528 0.12074781
 0.09755748 0.07023182 0.13788945 0.05630386 0.09610364 0.11178536
 0.12760187 0.12512233 0.10853237 0.09027881 0.19505063 0.12290816
 0.09531017 0.1067799  0.09913647 0.08609166 0.04781865 0.07623497
 0.09471651 0.06470144 0.07611287 0.077

### Run TD$(\lambda)$

In [12]:
N_iters = 1*10**5
v0 = np.zeros(N_s,dtype=float)
s0 = np.random.choice(N_s)
#step size
alpha_0 = 1.0
alpha = np.zeros(N_iters,dtype = float)
N_0 = 2*10**4
#powers = np.array([0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9],dtype=float)
powers = np.array([0.55,0.6,0.65,0.7,0.75,0.8],dtype = float) #C = 300, N_0 = 2*10**4
#powers = np.array([0.8,0.825,0.85,0.875,0.9,0.925,0.95])# C = 2000, N_0 = 10**4
alpha = np.zeros((len(powers),N_iters),dtype=float)
for j in range(len(powers)):
    for i in range(N_iters):
        alpha[j][i] = 300.0/(N_0+i)**(powers[j])

In [13]:
print(Policy.shape)
print(Policy.sum(axis=0))

(10, 50)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]


In [14]:
def main_loop(j,alpha,s_0,N_traj):
    V_funcs = np.zeros((N_traj,N_s))
    J_0 = np.zeros((N_traj,N_s))
    J_1 = np.zeros((N_traj,N_s))
    Transient = np.zeros((N_traj,N_s))
    np.random.seed(2020+j)
    ###Main loop
    for i in range(N_traj):
        V = np.zeros(N_s,dtype=float)
        J0_cur = np.zeros(N_s,dtype=float)
        J1_cur = np.zeros(N_s,dtype=float)
        Transient_cur = V - theta_star
        s0 = s_0
        D_n = np.zeros((N_s,1),dtype = float)
        D_n[s0,0] = 1.0
        for N in range(N_iters):
            #sample action
            a = np.random.choice(N_a, 1, replace=True, p=Policy[:,s0])
            a=a[0]
            #sample next state
            s = np.random.choice(Inds_nz[s0,a], 1, replace=True, p=P[Inds_nz[s0,a],s0,a])
            s=s[0]
            #calculate A_bar
            sec_vec = np.zeros((1,N_s),dtype = float)
            sec_vec[0,s0] = 1.0
            sec_vec[0,s] -= gamma
            #calculate A_bar
            A_bar = np.dot(D_n,sec_vec)
            #calculate b_bar
            b_bar = R[a,s0]*D_n[:,0]
            #calculate TD(\lambda) update and J_0
            TD_update = -A_bar @ V + b_bar
            eps_bar = b_bar - A_bar@theta_star
            #calculate J1
            J1_cur = (np.eye(N_s) - alpha[j][N]*A_star_new)@J1_cur - alpha[j][N]*(A_bar-A_star_new)@J0_cur
            #calculate transient term
            Transient_cur = (np.eye(N_s) - alpha[j][N]*A_bar)@Transient_cur
            #calculate J0
            J0_cur = (np.eye(N_s) - alpha[j][N]*A_star_new)@J0_cur + alpha[j][N]*eps_bar
            #update V
            V = V + alpha[j][N]*TD_update
            #update eligibility traces
            D_n = (TD_lambda*gamma)*D_n
            D_n[s,0] = D_n[s,0]+1
            #add some noise
            D_n[:,0] += 0.1*np.random.randn(N_s)
            #update current state
            s0 = s
        #save value function
        V_funcs[i] = V
        #save J_0
        J_0[i] = J0_cur
        #save J_1
        J_1[i] = J1_cur
        #save transient term
        Transient[i] = Transient_cur
    return np.asarray([V_funcs,J_0,J_1,Transient])  

In [15]:
N_traj = 100
nbcores = multiprocessing.cpu_count()
trav = Pool(nbcores)
res_indep = trav.starmap(main_loop, [(j,alpha,s0,N_traj) for j in range (len(powers))])
trav.close()

In [16]:
res_indep = np.asarray(res_indep)
print(res_indep.shape)

(6, 4, 100, 50)


In [17]:
norms = np.zeros((len(powers),N_iters),dtype=float)
norms_J0_rem = np.zeros((len(powers),N_iters),dtype=float)
norms_J1_rem = np.zeros((len(powers),N_iters),dtype=float)
norms_transient = np.zeros((len(powers),N_iters),dtype=float)

norms_J0 = np.zeros((len(powers),N_traj),dtype=float)
norms_J1 = np.zeros((len(powers),N_traj),dtype=float)
norms_H0 = np.zeros((len(powers),N_traj),dtype=float)
norms_H1 = np.zeros((len(powers),N_traj),dtype=float)
for j in range(len(powers)):
    for i in range(N_traj):
        #norms[j][i] = np.linalg.norm(res_indep[j,0,i,:]-theta_star)
        #norms_J0_rem[j][i] = np.linalg.norm(res_indep[j,0,i,:] - res_indep[j,1,i,:]-theta_star)
        #norms_J1_rem[j][i] = np.linalg.norm(res_indep[j,0,i,:] - res_indep[j,1,i,:]-res_indep[j,2,i,:]-theta_star)
        #norms_transient[j][i] = np.linalg.norm(res_indep[j,3,i,:])
        norms_J0[j][i] = np.linalg.norm(res_indep[j,1,i,:])
        norms_J1[j][i] = np.linalg.norm(res_indep[j,2,i,:])
        norms_H0[j][i] = np.linalg.norm(res_indep[j,0,i,:] - res_indep[j,1,i,:]-theta_star-res_indep[j,3,i,:])
        norms_H1[j][i] = np.linalg.norm(res_indep[j,0,i,:] - res_indep[j,1,i,:]-theta_star-res_indep[j,3,i,:]-res_indep[j,2,i,:])

In [18]:
import matplotlib.pyplot as plt
N_start = 0*10**5
N_last = 2*10**5
j = 3
plt.figure(figsize=(12,8)) 
plt.plot(np.arange(N_start,N_last), norms[j][N_start:N_last], color='r' ,label='MSE error, TD(0.9)') 
#plt.plot(np.arange(N_start,N_last), norms_1[j][N_start:N_last], color='g' ,label='MSE error, TD(0.9)') 
plt.plot(np.arange(N_start,N_iters), norms_J0_rem[j][N_start:], color='g' ,label='MSE error without J_0') 
plt.plot(np.arange(N_start,N_iters), norms_J1_rem[j][N_start:], color='b' ,label='MSE error without J_0, J_1')
#plt.xlabel('iteration number',fontsize = 18)
#plt.ylabel('cost',fontsize = 18) 
#plt.title('VR cost for MDCV, Gaussian distribution, quadratic target',fontsize = 20)
plt.yscale('log')
plt.grid(linestyle='--', linewidth=1.0)
#plt.legend() 
plt.show()
#plt.savefig("pics/TD_lambda_MSE.pdf")

ValueError: x and y must have same first dimension, but have shapes (200000,) and (100000,)

### Save results

In [ ]:
import os
if not os.path.exists("results_09_06_TD(lambda)_gamma_05"):
    os.makedirs("results_09_06_TD(lambda)_gamma_05")

In [ ]:
np.save("results_09_05_TD(lambda)_gamma_05/all_res.npy",res_indep)
#np.save("results_31_05_TD(0)_gamma_05/TD0_error.npy",norms)
#np.save("results_31_05_TD(0)_gamma_05/TD0_J0_remainder.npy",norms_J0_rem)
#np.save("results_31_05_TD(0)_gamma_05/TD0_J1_remainder.npy",norms_J1_rem)
#np.save("results_31_05_TD(0)_gamma_05/Transient.npy",norms_transient)
np.save("results_09_06_TD(lambda)_gamma_05/norms_J0.npy",norms_J0)
np.save("results_09_06_TD(lambda)_gamma_05/norms_J1.npy",norms_J1)
np.save("results_09_06_TD(lambda)_gamma_05/norms_H0.npy",norms_H0)
np.save("results_09_06_TD(lambda)_gamma_05/norms_H1.npy",norms_H1)

### Plot graphics

In [ ]:
import matplotlib.pyplot as plt
N_start = 2*10**4
j = 3
plt.figure(figsize=(12,8)) 
#plt.plot(np.arange(N_start,N_iters), norms[j][N_start:], color='r' ,label='MSE error') 
plt.plot(np.arange(N_start,N_iters), norms_J0[j][N_start:], color='m' ,label='norm J_0') 
plt.plot(np.arange(N_start,N_iters), norms_H0[j][N_start:], color='g' ,label='norm H_0') 

plt.plot(np.arange(N_start,N_iters), norms_J1[j][N_start:], color='b' ,label='norm J_1')
plt.plot(np.arange(N_start,N_iters), norms_H1[j][N_start:], color='r' ,label='norm H_1')

#plt.plot(np.arange(N_start,N_iters), norms_transient[j][N_start:], color='k' ,label='transient term')
#plt.xlabel('iteration number',fontsize = 18)
#plt.ylabel('cost',fontsize = 18) 
#plt.title('VR cost for MDCV, Gaussian distribution, quadratic target',fontsize = 20)
plt.yscale('log')
plt.grid(linestyle='--', linewidth=1.0)
#plt.legend() 
plt.show()
#plt.savefig("pics/TD_lambda_J_0_H_0_J_1_H_1.pdf")

### Calculate mean statistics

In [ ]:
mean_norm_J0 = np.zeros(len(powers),dtype=float)
mean_norm_J1 = np.zeros(len(powers),dtype = float)

mean_norm_H0 = np.zeros(len(powers),dtype=float)
mean_norm_H1 = np.zeros(len(powers),dtype=float)

N_start = 0

for j in range(len(powers)):
    mean_norm_J0[j] = np.mean(norms_J0[j][N_start:]**2)
    mean_norm_J1[j] = np.mean(norms_J1[j][N_start:]**2)
    mean_norm_H0[j] = np.mean(norms_H0[j][N_start:]**2)
    mean_norm_H1[j] = np.mean(norms_H1[j][N_start:]**2)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,8)) 
#plt.plot(powers, mean_norm_J0, linestyle='--', marker='o', color='r', label='J_0')
#plt.plot(powers, 300.0/(N_0+10**5)**powers, linestyle='--', marker='o', color='g', label='predicted J_0')
plt.plot(powers, mean_norm_J1, linestyle='--', marker='o', color='b',label='J_1') 
plt.plot(powers, (300.0/(N_0+10**5)**powers)**((3*powers-1)/powers), marker='o', color='g',label='J_1')
#plt.plot(powers, np.exp(1-3*powers), linestyle='--', marker='o', color='g',label='J_1')
#plt.plot(powers, mean_norm_H0, linestyle='--', marker='o', color='b', label='H_0') 
#plt.plot(powers, mean_norm_H1, linestyle='--', marker='o', color='m',label='H_1')
plt.xlabel('iteration number',fontsize = 18)
#plt.ylabel('cost',fontsize = 18) 
#plt.title('VR cost for MDCV, Gaussian distribution, quadratic target',fontsize = 20)
plt.yscale('log')
plt.legend() 
plt.show()

In [ ]:
slope_J0, intercept_J0, r_value, p_value, std_err = stats.linregress(np.ones(len(powers)),np.log(mean_norm_J0) + powers)
#slope_J1, intercept_J1, r_value, p_value, std_err = stats.linregress(powers,np.log(mean_norm_J1))
#slope_H0, intercept_H0, r_value, p_value, std_err = stats.linregress(powers,np.log(mean_norm_H0))
#slope_H1, intercept_H1, r_value, p_value, std_err = stats.linregress(powers,np.log(mean_norm_H1))
print(slope_J0)
print(intercept_J0)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,8)) 
#J_0
plt.plot(powers, mean_norm_J0, linestyle='--', marker='o', color='r', label='J_0') 
#plt.plot(powers, np.exp(slope_J0*powers + intercept_J0), linestyle='--', marker='o', color='m',label='regressed J_0') 
#J_1
plt.plot(powers, mean_norm_J1, linestyle='--', marker='o', color='g', label='J_1') 
#plt.plot(powers, np.exp(slope_J1*powers + intercept_J1), linestyle='--', marker='o', color='b',label='regressed J_1')
#H_1
plt.plot(powers, mean_norm_H1, linestyle='--', marker='o', color='b', label='H_1') 
#plt.plot(powers, np.exp(slope_H1*powers + intercept_H1), linestyle='--', marker='o', color='y',label='regressed H_1')
#plt.xlabel('iteration number',fontsize = 18)
#plt.ylabel('cost',fontsize = 18) 
#plt.title('VR cost for MDCV, Gaussian distribution, quadratic target',fontsize = 20)
plt.yscale('log')
plt.grid(linestyle='--', linewidth=1.0)
plt.legend() 
#plt.show()
plt.savefig("pics/TD_lambda_J_0_J_1_H_1_gamma_08.pdf")